In [98]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
#from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import mean_squared_error

In [99]:
#df = pd.read_csv("reviewsWithHeader.csv")
df = pd.read_json('C:/Users/Dhruv/Desktop/Semester3/PROJECT 2018/Recommendation system/Data/Beauty5.json',lines=True)
df.head()


,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,7806397051,"[3, 4]",1,Very oily and creamy. Not at all what I expect...,"01 30, 2014",A1YJEY40YUW4SE,Andrea,Don't waste your money,1391040000
1,7806397051,"[1, 1]",3,This palette was a decent price and I was look...,"04 18, 2014",A60XNB876KYML,Jessica H.,OK Palette!,1397779200
2,7806397051,"[0, 1]",4,The texture of this concealer pallet is fantas...,"09 6, 2013",A3G6XNM240RMWA,Karen,great quality,1378425600
3,7806397051,"[2, 2]",2,I really can't tell what exactly this thing is...,"12 8, 2013",A1PQFP6SAJ6D80,Norah,Do not work on my face,1386460800
4,7806397051,"[0, 0]",3,"It was a little smaller than I expected, but t...","10 19, 2013",A38FVHZTNQ271F,Nova Amor,It's okay.,1382140800


In [117]:
dfdistint = df.asin.unique()
print(df.columns)
print(df.shape)
print(dfdistint.shape)

Index(['asin', 'helpful', 'overall', 'reviewText', 'reviewTime', 'reviewerID',
       'reviewerName', 'summary', 'unixReviewTime'],
      dtype='object')
(477179, 9)
(35123,)


### Product based Collaborative Filtering

In [113]:
count = df.groupby("asin", as_index=False).count()
mean = df.groupby("asin", as_index=False).mean()

print(count.shape)
print(mean.head(10))

(35123, 9)
         asin   overall  unixReviewTime
0  0000031887  4.608696    1.370064e+09
1  0123456479  4.166667    1.382947e+09
2  1608299953  4.307692    1.307637e+09
3  1617160377  4.625000    1.321078e+09
4  7806397051  2.625000    1.382087e+09
5  9759091062  3.090909    1.390930e+09
6  9788072216  5.000000    1.342552e+09
7  9790790961  4.333333    1.378858e+09
8  9790794231  3.600000    1.298212e+09
9  B00001W0KA  4.625000    1.349492e+09


In [102]:
dfMerged = pd.merge(df, count, how='right', on=['asin'])
print(dfMerged.shape)
dfMerged

(477179, 17)


,asin,helpful_x,overall_x,reviewText_x,reviewTime_x,reviewerID_x,reviewerName_x,summary_x,unixReviewTime_x,helpful_y,overall_y,reviewText_y,reviewTime_y,reviewerID_y,reviewerName_y,summary_y,unixReviewTime_y
0,7806397051,"[3, 4]",1,Very oily and creamy. Not at all what I expect...,"01 30, 2014",A1YJEY40YUW4SE,Andrea,Don't waste your money,1391040000,8,8,8,8,8,8,8,8
1,7806397051,"[1, 1]",3,This palette was a decent price and I was look...,"04 18, 2014",A60XNB876KYML,Jessica H.,OK Palette!,1397779200,8,8,8,8,8,8,8,8
2,7806397051,"[0, 1]",4,The texture of this concealer pallet is fantas...,"09 6, 2013",A3G6XNM240RMWA,Karen,great quality,1378425600,8,8,8,8,8,8,8,8
3,7806397051,"[2, 2]",2,I really can't tell what exactly this thing is...,"12 8, 2013",A1PQFP6SAJ6D80,Norah,Do not work on my face,1386460800,8,8,8,8,8,8,8,8
4,7806397051,"[0, 0]",3,"It was a little smaller than I expected, but t...","10 19, 2013",A38FVHZTNQ271F,Nova Amor,It's okay.,1382140800,8,8,8,8,8,8,8,8
5,7806397051,"[1, 2]",5,"I was very happy to get this palette, now I wi...","04 15, 2013",A3BTN14HIZET6Z,"S. M. Randall ""WildHorseWoman""",Very nice palette!,1365984000,8,8,8,8,8,8,8,8
6,7806397051,"[1, 3]",1,PLEASE DONT DO IT! this just rachett the palet...,"08 16, 2013",A1Z59RFKN0M5QL,"tasha ""luvely12b""",smh!!!,1376611200,8,8,8,8,8,8,8,8
7,7806397051,"[0, 1]",2,"Chalky,Not Pigmented,Wears off easily,Not a Co...","09 4, 2013",AWUO9P6PL1SY8,TreMagnifique,"Chalky, Not Pigmented, Wears off easily, Not a...",1378252800,8,8,8,8,8,8,8,8
8,9759091062,"[0, 0]",2,Did nothing for me. Stings when I put it on. I...,"07 13, 2014",A3LMILRM9OC3SA,NaN,"no Lightening, no Brightening,......NOTHING",1405209600,11,11,11,11,11,10,11,11
9,9759091062,"[0, 0]",3,I bought this product to get rid of the dark s...,"12 27, 2013",A30IP88QK3YUIO,Amina Bint Ibraheem,Its alright,1388102400,11,11,11,11,11,10,11,11


In [103]:
#rename column
dfMerged["totalReviewers"] = dfMerged["reviewerID_y"]
dfMerged["overallScore"] = dfMerged["overall_x"]
dfMerged["summaryReview"] = dfMerged["summary_x"]

dfNew = dfMerged[['asin','summaryReview','overallScore',"totalReviewers"]]
print(dfNew.shape)
dfNew.head(10)

(477179, 4)


,asin,summaryReview,overallScore,totalReviewers
0,7806397051,Don't waste your money,1,8
1,7806397051,OK Palette!,3,8
2,7806397051,great quality,4,8
3,7806397051,Do not work on my face,2,8
4,7806397051,It's okay.,3,8
5,7806397051,Very nice palette!,5,8
6,7806397051,smh!!!,1,8
7,7806397051,"Chalky, Not Pigmented, Wears off easily, Not a...",2,8
8,9759091062,"no Lightening, no Brightening,......NOTHING",2,11
9,9759091062,Its alright,3,11


Selecting products which have more than 50 reviews

In [104]:
print(dfMerged.shape)
dfMerged = dfMerged.sort_values(by='totalReviewers', ascending=False)
dfCount = dfMerged[dfMerged.totalReviewers >= 100]
dfCount.shape

(477179, 20)


(39731, 20)

In [119]:
dfdistint = dfMerged.asin.unique()
dfdistint.shape

(35123,)

### Grouping all the summary Reviews by product ID

In [105]:
dfProductReview = df.groupby("asin", as_index=False).mean()                            
print(dfProductReview.shape)
ProductReviewSummary = dfCount.groupby("asin")["summaryReview"].apply(list)          # 
print((ProductReviewSummary))
ProductReviewSummary = pd.DataFrame(ProductReviewSummary)
#print(ProductReviewSummary)
ProductReviewSummary.to_csv("ProductReviewSummary.csv")

(35123, 3)
asin
B0000530ED    [Review, Best alternative to bleaching., Look ...
B0000632EN    [Love it! (but you have to wear other moisturi...
B000072UMJ    [like they should be, Save the trip to the mal...
B0000ANHST    [It's a carhartt what more can you say, Good L...
B0000C321X    [Not what I expected, heavy, durable fabric. w...
B0000CC64W    [Excellent serum, Good deal for the money, SIM...
B00012O12A    [Classic Levi Product - Always a Hit!, For the...
B000142C1A    [Great Product Quality, Natural and neat, Prin...
B000142FVW    [Nice, Love this stuff!, Great color/Great pol...
B00016XJ4M    [Perfect, Great toner!, Great toner!, Alcohol ...
B0001ZNZJM    [Great, but be aware of riding up!, Comfortabl...
B0002TOZ1E    [Get the larger size., Still great after all t...
B0002Z8HAI    [Beautiful color, Five Stars, if you like purp...
B0006LMBJ6    [PERFECT, perfect, good, levis 511 men skinny ...
B0007CKMA4    [Great Fit and Color; Well-made!, big&tall jea...
B0007QCOTM    [Good!, Ve

In [106]:
dfProductReview.head(10)

,asin,overall,unixReviewTime
0,0000031887,4.608696,1.370064e+09
1,0123456479,4.166667,1.382947e+09
2,1608299953,4.307692,1.307637e+09
3,1617160377,4.625000,1.321078e+09
4,7806397051,2.625000,1.382087e+09
5,9759091062,3.090909,1.390930e+09
6,9788072216,5.000000,1.342552e+09
7,9790790961,4.333333,1.378858e+09
8,9790794231,3.600000,1.298212e+09
9,B00001W0KA,4.625000,1.349492e+09


### create dataframe with certain columns

In [115]:
df3 = pd.read_csv("ProductReviewSummary.csv")
print(df3.shape)
df3 = pd.merge(df3, dfProductReview, on="asin", how='inner')
print(df3.head(10))

(266, 2)
         asin                                      summaryReview   overall  \
0  B0000530ED  ['Review', 'Best alternative to bleaching.', '...  4.009709   
1  B0000632EN  ['Love it! (but you have to wear other moistur...  3.802721   
2  B000072UMJ  ['like they should be', 'Save the trip to the ...  4.594595   
3  B0000ANHST  ["It's a carhartt what more can you say", 'Goo...  4.487179   
4  B0000C321X  ['Not what I expected', 'heavy, durable fabric...  4.263415   
5  B0000CC64W  ['Excellent serum', 'Good deal for the money',...  4.314685   
6  B00012O12A  ['Classic Levi Product - Always a Hit!', 'For ...  4.153439   
7  B000142C1A  ['Great Product Quality', 'Natural and neat', ...  4.482456   
8  B000142FVW  ['Nice', 'Love this stuff!', 'Great color/Grea...  4.552980   
9  B00016XJ4M  ['Perfect', 'Great toner!', 'Great toner!', 'A...  4.485981   

   unixReviewTime  
0    1.364436e+09  
1    1.364625e+09  
2    1.368799e+09  
3    1.371795e+09  
4    1.363199e+09  
5    1.34406

In [108]:
df3 = df3[['asin','summaryReview','overall']]
df3.head(10)

,asin,summaryReview,overall
0,B0000530ED,"['Review', 'Best alternative to bleaching.', '...",4.009709
1,B0000632EN,['Love it! (but you have to wear other moistur...,3.802721
2,B000072UMJ,"['like they should be', 'Save the trip to the ...",4.594595
3,B0000ANHST,"[""It's a carhartt what more can you say"", 'Goo...",4.487179
4,B0000C321X,"['Not what I expected', 'heavy, durable fabric...",4.263415
5,B0000CC64W,"['Excellent serum', 'Good deal for the money',...",4.314685
6,B00012O12A,"['Classic Levi Product - Always a Hit!', 'For ...",4.153439
7,B000142C1A,"['Great Product Quality', 'Natural and neat', ...",4.482456
8,B000142FVW,"['Nice', 'Love this stuff!', 'Great color/Grea...",4.552980
9,B00016XJ4M,"['Perfect', 'Great toner!', 'Great toner!', 'A...",4.485981


### Text Cleaning - Summary column

In [110]:
#function for tokenizing summary
regEx = re.compile('[^a-z]+')
def cleanReviews(reviewText):
    reviewText = reviewText.lower()
    reviewText = regEx.sub(' ', reviewText).strip()
    return reviewText

In [111]:
#reset index and drop duplicate rows
df3["summaryClean"] = df3["summaryReview"].apply(cleanReviews)
df3 = df3.drop_duplicates(['overall'], keep='last')
df3 = df3.reset_index()

print(df3.shape)
df3.head(10)

(261, 5)


,index,asin,summaryReview,overall,summaryClean
0,0,B0000530ED,"['Review', 'Best alternative to bleaching.', '...",4.009709,review best alternative to bleaching look else...
1,1,B0000632EN,['Love it! (but you have to wear other moistur...,3.802721,love it but you have to wear other moisturizer...
2,3,B0000ANHST,"[""It's a carhartt what more can you say"", 'Goo...",4.487179,it s a carhartt what more can you say good loo...
3,4,B0000C321X,"['Not what I expected', 'heavy, durable fabric...",4.263415,not what i expected heavy durable fabric waist...
4,5,B0000CC64W,"['Excellent serum', 'Good deal for the money',...",4.314685,excellent serum good deal for the money simply...
5,6,B00012O12A,"['Classic Levi Product - Always a Hit!', 'For ...",4.153439,classic levi product always a hit for the pric...
6,7,B000142C1A,"['Great Product Quality', 'Natural and neat', ...",4.482456,great product quality natural and neat princes...
7,8,B000142FVW,"['Nice', 'Love this stuff!', 'Great color/Grea...",4.552980,nice love this stuff great color great polish ...
8,9,B00016XJ4M,"['Perfect', 'Great toner!', 'Great toner!', 'A...",4.485981,perfect great toner great toner alcohol free n...
9,10,B0001ZNZJM,"['Great, but be aware of riding up!', 'Comfort...",4.472081,great but be aware of riding up comfortable an...


In [78]:
reviews = df3["summaryClean"] 
countVector = CountVectorizer(max_features = 300, stop_words='english') 
transformedReviews = countVector.fit_transform(reviews) 
print(transformedReviews.A.shape)
dfReviews = DataFrame(transformedReviews.A, columns=countVector.get_feature_names())
dfReviews = dfReviews.astype(int)
dfReviews.head(10)

(261, 300)


,absolutely,acne,adorable,adovia,amazing,argan,art,awesome,bad,base,...,women,wonderful,work,worked,working,works,worth,wow,wrong,yes
0,0,0,0,0,0,0,0,1,1,0,...,0,1,0,1,0,1,0,0,0,0
1,0,3,0,0,0,0,0,0,1,0,...,0,1,3,0,0,3,2,1,0,2
2,0,0,0,0,0,0,0,0,1,0,...,0,1,2,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,3,1,0,...,1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,0,0,3,0,...,0,3,3,0,0,9,1,0,0,0
5,0,0,0,0,0,0,0,2,1,0,...,1,1,0,0,0,0,0,0,2,0
6,0,0,0,0,2,0,0,2,0,0,...,0,1,0,0,0,0,0,0,0,0
7,0,0,0,0,2,0,0,2,1,0,...,1,0,0,0,0,0,1,2,1,1
8,0,2,0,0,4,0,0,3,0,0,...,0,6,0,0,0,5,0,0,0,2
9,0,0,0,0,1,0,0,6,0,0,...,0,0,0,1,0,1,3,3,2,0


In [79]:
#save 
dfReviews.to_csv("dfReviews.csv")

In [80]:
# First let's create a dataset called X
X = np.array(dfReviews)
 # create train and test
tpercent = 0.9
tsize = int(np.floor(tpercent * len(dfReviews)))
dfReviews_train = X[:tsize]
dfReviews_test = X[tsize:]
#len of train and test
lentrain = len(dfReviews_train)
lentest = len(dfReviews_test)

In [83]:
# KNN classifier to find similar products
print(lentrain)
print(lentest)

234
27


In [86]:
neighbor = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(dfReviews_train)

# Let's find the k-neighbors of each point in object X. To do that we call the kneighbors() function on object X.
distances, indices = neighbor.kneighbors(dfReviews_train)
print(distances,indices)

[[ 0.         31.38470965 31.41655614]
 [ 0.         29.3257566  29.47880595]
 [ 0.         29.9165506  41.35214626]
 [ 0.         22.3383079  28.40774542]
 [ 0.         26.43860813 29.22327839]
 [ 0.         22.3383079  30.        ]
 [ 0.         21.26029163 22.97825059]
 [ 0.         75.65051223 87.21238444]
 [ 0.         48.59012245 59.9082632 ]
 [ 0.         32.54228019 33.43650699]
 [ 0.         24.71841419 26.87005769]
 [ 0.         20.34698995 30.24896692]
 [ 0.         24.97999199 32.41913015]
 [ 0.         23.40939982 23.60084744]
 [ 0.         20.04993766 22.36067977]
 [ 0.         32.28002478 32.31098884]
 [ 0.         24.97999199 27.25802634]
 [ 0.         18.30300522 21.56385865]
 [ 0.         23.19482701 26.92582404]
 [ 0.         43.43961326 50.11985634]
 [ 0.         21.74856317 25.94224354]
 [ 0.         16.46207763 21.54065923]
 [ 0.         49.74937186 69.33974329]
 [ 0.         25.96150997 27.01851217]
 [ 0.         25.03996805 26.        ]
 [ 0.         21.81742423

In [18]:
#find most related products
for i in range(lentest):
    a = neighbor.kneighbors([dfReviews_test[i]])
    related_product_list = a[1]

    first_related_product = [item[0] for item in related_product_list]
    first_related_product = str(first_related_product).strip('[]')
    first_related_product = int(first_related_product)
    second_related_product = [item[1] for item in related_product_list]
    second_related_product = str(second_related_product).strip('[]')
    second_related_product = int(second_related_product)
    
    print ("Based on product reviews, for ", df3["asin"][lentrain + i] ," average rating is ",df3["overall"][lentrain + i])
    print ("The first similar product is ", df3["asin"][first_related_product] ," average rating is ",df3["overall"][first_related_product])
    print ("The second similar product is ", df3["asin"][second_related_product] ," average rating is ",df3["overall"][second_related_product])
    print ("-----------------------------------------------------------")

Based on product reviews, for  B00AO4EMKQ  average rating is  4.285714285714286
The first similar product is  B005TI7LHS  average rating is  4.139130434782609
The second similar product is  B00AO4EL34  average rating is  4.1909090909090905
-----------------------------------------------------------
Based on product reviews, for  B00AWLB9G6  average rating is  4.39622641509434
The first similar product is  B00AO4E9MC  average rating is  4.339130434782609
The second similar product is  B00AE078YM  average rating is  3.78
-----------------------------------------------------------
Based on product reviews, for  B00AWLB9I4  average rating is  4.207920792079208
The first similar product is  B00AE078YM  average rating is  3.78
The second similar product is  B00AO4E9MC  average rating is  4.339130434782609
-----------------------------------------------------------
Based on product reviews, for  B00BNB3A0W  average rating is  3.4414414414414414
The first similar product is  B009T47YZ2  averag

In [19]:
print ("Based on product reviews, for ", df3["asin"][260] ," average rating is ",df3["overall"][260])
print ("The first similar product is ", df3["asin"][first_related_product] ," average rating is ",df3["overall"][first_related_product])
print ("The second similar product is ", df3["asin"][second_related_product] ," average rating is ",df3["overall"][second_related_product])
print ("-----------------------------------------------------------")

Based on product reviews, for  B00KQBR9FM  average rating is  4.745098039215686
The first similar product is  B00AO4EKSU  average rating is  4.095652173913043
The second similar product is  B0030HKJ8I  average rating is  4.324137931034483
-----------------------------------------------------------


### Predicting Review Score

In [20]:
df5_train_target = df3["overall"][:lentrain]
df5_test_target = df3["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 3
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)

print(classification_report(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           3       0.33      0.33      0.33         3
           4       0.92      0.92      0.92        24

   micro avg       0.85      0.85      0.85        27
   macro avg       0.62      0.62      0.62        27
weighted avg       0.85      0.85      0.85        27



### Accuracy of the model

In [21]:
print (accuracy_score(df5_test_target, knnpreds_test))

0.8518518518518519


In [22]:
print(mean_squared_error(df5_test_target, knnpreds_test))

0.14814814814814814


### Predicting Review Score with k = 5

In [23]:
df5_train_target = df3["overall"][:lentrain]
df5_test_target = df3["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 5
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)
#print (knnpreds_test)

print(classification_report(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           3       1.00      0.33      0.50         3
           4       0.92      1.00      0.96        24

   micro avg       0.93      0.93      0.93        27
   macro avg       0.96      0.67      0.73        27
weighted avg       0.93      0.93      0.91        27



In [24]:
print (accuracy_score(df5_test_target, knnpreds_test))

0.9259259259259259


In [25]:
print(mean_squared_error(df5_test_target, knnpreds_test))

0.07407407407407407


### Predicting reviews with 85, 15 train, test split and k = 5

In [26]:
# First let's create a dataset called X
X = np.array(dfReviews)
 # create train and test
tpercent = 0.85
tsize = int(np.floor(tpercent * len(dfReviews)))
dfReviews_train = X[:tsize]
dfReviews_test = X[tsize:]
#len of train and test
lentrain = len(dfReviews_train)
lentest = len(dfReviews_test)

In [27]:
# Next we will instantiate a nearest neighbor object, and call it nbrs. Then we will fit it to dataset X.
neighbor = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(dfReviews_train)

# Let's find the k-neighbors of each point in object X. To do that we call the kneighbors() function on object X.
distances, indices = neighbor.kneighbors(dfReviews_train)

In [28]:
#find most related products
for i in range(lentest):
    a = neighbor.kneighbors([dfReviews_test[i]])
    related_product_list = a[1]

    first_related_product = [item[0] for item in related_product_list]
    first_related_product = str(first_related_product).strip('[]')
    first_related_product = int(first_related_product)
    second_related_product = [item[1] for item in related_product_list]
    second_related_product = str(second_related_product).strip('[]')
    second_related_product = int(second_related_product)
    
    print ("Based on product reviews, for ", df3["asin"][lentrain + i] ," average rating is ",df3["overall"][lentrain + i])
    print ("The first similar product is ", df3["asin"][first_related_product] ," average rating is ",df3["overall"][first_related_product])
    print ("The second similar product is ", df3["asin"][second_related_product] ," average rating is ",df3["overall"][second_related_product])
    print ("-----------------------------------------------------------")

Based on product reviews, for  B00AE0790U  average rating is  4.1457286432160805
The first similar product is  B007RTR9DS  average rating is  2.9884393063583814
The second similar product is  B002WTC37A  average rating is  4.3097345132743365
-----------------------------------------------------------
Based on product reviews, for  B00AE07BDU  average rating is  4.217821782178218
The first similar product is  B005XIDZHO  average rating is  3.4554455445544554
The second similar product is  B0056GDG90  average rating is  4.017543859649122
-----------------------------------------------------------
Based on product reviews, for  B00AE07FQI  average rating is  3.8316831683168315
The first similar product is  B005XIDZHO  average rating is  3.4554455445544554
The second similar product is  B001LF4I8I  average rating is  4.2407407407407405
-----------------------------------------------------------
Based on product reviews, for  B00AE07H0M  average rating is  3.875
The first similar product is

In [29]:
df5_train_target = df3["overall"][:lentrain]
df5_test_target = df3["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)

n_neighbors = 5
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)
#print (knnpreds_test)

print(classification_report(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           3       0.33      0.20      0.25         5
           4       0.89      0.94      0.92        35

   micro avg       0.85      0.85      0.85        40
   macro avg       0.61      0.57      0.58        40
weighted avg       0.82      0.85      0.83        40



In [30]:
print (accuracy_score(df5_test_target, knnpreds_test))

0.85


In [31]:
print(mean_squared_error(df5_test_target, knnpreds_test))

0.15


### kNN with k = 3, Algorithm = Brute

In [32]:
neighbor = NearestNeighbors(n_neighbors=3, algorithm='brute').fit(dfReviews_train)

distances, indices = neighbor.kneighbors(dfReviews_train)


In [33]:
df5_train_target = df3["overall"][:lentrain]
df5_test_target = df3["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)
n_neighbors = 3
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)

print(classification_report(df5_test_target, knnpreds_test))
print ("Accuracy: ",accuracy_score(df5_test_target, knnpreds_test))
print("MSE: ",mean_squared_error(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00         0
           3       0.33      0.20      0.25         5
           4       0.89      0.91      0.90        35

   micro avg       0.82      0.82      0.82        40
   macro avg       0.41      0.37      0.38        40
weighted avg       0.82      0.82      0.82        40

Accuracy:  0.825
MSE:  0.25


C:\Users\Dhruv\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### kNN with k= 5, and Algorithm = KD_Tree

In [34]:
neighbor = NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(dfReviews_train)
distances, indices = neighbor.kneighbors(dfReviews_train)

In [35]:
df5_train_target = df3["overall"][:lentrain]
df5_test_target = df3["overall"][lentrain:lentrain+lentest]
df5_train_target = df5_train_target.astype(int)
df5_test_target = df5_test_target.astype(int)


n_neighbors = 5
knnclf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance')
knnclf.fit(dfReviews_train, df5_train_target)
knnpreds_test = knnclf.predict(dfReviews_test)

print(classification_report(df5_test_target, knnpreds_test))
print ("Accuracy: ",accuracy_score(df5_test_target, knnpreds_test))
print("MSE: ",mean_squared_error(df5_test_target, knnpreds_test))

              precision    recall  f1-score   support

           3       0.33      0.20      0.25         5
           4       0.89      0.94      0.92        35

   micro avg       0.85      0.85      0.85        40
   macro avg       0.61      0.57      0.58        40
weighted avg       0.82      0.85      0.83        40

Accuracy:  0.85
MSE:  0.15


### Word correltation based on cluster

In [36]:
cluster = df.groupby("overall")["summary"].apply(list)
cluster = pd.DataFrame(cluster)
cluster.to_csv("cluster.csv")
cluster1 = pd.read_csv("cluster.csv")
cluster1["summaryClean"] = cluster1["summary"].apply(cleanReviews)

### World Clouding for Each Score Group

In [37]:
'''
stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=500,
        max_font_size=30, 
        scale=3,
        random_state=1 # chosen at random by flipping a coin; it was heads
    ).generate(str(data))
    
    fig = plt.figure(1, figsize=(8, 8))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()
'''

"\nstopwords = set(STOPWORDS)\n\ndef show_wordcloud(data, title = None):\n    wordcloud = WordCloud(\n        background_color='white',\n        stopwords=stopwords,\n        max_words=500,\n        max_font_size=30, \n        scale=3,\n        random_state=1 # chosen at random by flipping a coin; it was heads\n    ).generate(str(data))\n    \n    fig = plt.figure(1, figsize=(8, 8))\n    plt.axis('off')\n    if title: \n        fig.suptitle(title, fontsize=20)\n        fig.subplots_adjust(top=2.3)\n\n    plt.imshow(wordcloud)\n    plt.show()\n"

In [38]:
#show_wordcloud(cluster1["summaryClean"][0], title = "Review Score One")

In [39]:
#show_wordcloud(cluster1["summaryClean"][1] , title = "Review Score Two")

In [40]:
#show_wordcloud(cluster1["summaryClean"][2], title = "Review Score Three")

In [41]:
#show_wordcloud(cluster1["summaryClean"][3], title = "Review Score Four")

In [42]:
#show_wordcloud(cluster1["summaryClean"][4], title = "Review Score Five")